<a href="https://colab.research.google.com/github/HernanJMaurice/androidSensors/blob/main/HPC/Maurice_Hernan_Ejercicio_1_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introduccion

  Una secuencia de ADN, secuencia de nucleótidos o secuencia genética es una sucesión de letras representando la estructura primaria de una molécula real o hipotética de ADN o banda, con la capacidad de transportar información.

  Las letras son A, C, G y T, que simbolizan las cuatro subunidades de nucleótidos de una banda ADN - adenina, citosina, guanina, timina, que son bases covalentemente ligadas a cadenas fosfóricas. En el típico caso, las secuencias se presentan pegadas unas a las otras, sin espacios, como en la secuencia AAAGTCTGAC, yendo de 5' a 3' de izquierda a derecha.

  La doble hélice del ADN, funciona para que los nucleótidos de cada cadena de ADN no sean exactamente iguales: en su lugar, son cadenas complementarias con bases de nucleótidos complementarios. Esto es, cada nucleótido es siempre emparejado con otro nucleótido específico en la cadena opuesta. Específicamente, la adenina (A) siempre estará complementada con la tiamina (T) en la cadena opuesta y la citosina (C) con la guanina (G) en la cadena opuesta.

  Para obtener la cadena complementaria se debe verificar letra por letra de una cadena y emparejarlas con su opuesta. 
  Este Programa permite realizar el calculo de cadenas complementarias y cadenas complementarias inversas a partir de una cadena de nucleotidos. 
  
  Recuerde que los nucleotidos aceptados son: A, C, G, T.

  Ejemplo:
   Cadena: CGATCCTAT
   Cadena complementaria: GCTAGGATA
   Cadena Complentaria inversa: ATAGGATCG

Armado del ambiente: En esta sección deben estar todos los comandos previos, que son necesarios para la ejecución del desarrollo.
# 2 Armado del Ambiente

Se debe instalar el modulo de CUDA de Python.

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 19.0MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
     |████████████████████████████████| 81kB 11.7MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=620990 sha256=81c849feb271aad3b93682b634bcf69e1c2b77cd5fe0697d1ffe9c8510208462
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=04f0b265780d9446176288705cdac389d19a198f16a7261c0313be0fce985d09
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


# 3 Desarrollo


In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: false }

cadena_ADN = 'ACTGAAATCGCGGC'#@param {type: "string"}

try:
    # Chequeo que solo tenga Nucleotidos permitidos.
    if not cadena_ADN:
        raise Exception("Error:")
except Exception as e:
    print( e.args, "Ingrese una cadena de ADN." )   # Muestra el parámetros.


# Lo paso siempre a Mayuscula.
cadena_ADN = cadena_ADN.upper()

import re
import numpy

pattern = re.compile("(A|C|T|G)+")


try:
    # Chequeo que solo tenga Nucleotidos permitidos.
    if pattern.fullmatch(cadena_ADN) is None :
        raise Exception("Error:")
except Exception as e:
    print( e.args, "Los nucleotidos pueden ser: A, C, T o G." )   # Muestra el parámetros.

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


#Creo Array de CHAR
cadena_ADN_char = numpy.array(list(cadena_ADN))

cantidad_Nucleotidos = len(cadena_ADN_char)

cadena_ADN_Complementaria = numpy.empty_like(cadena_ADN_char)
cadena_ADN_Complementaria_Inversa  = numpy.empty_like(cadena_ADN_char)

# --------------------------------------------

from datetime import datetime

tiempo_total = datetime.now()

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# CPU - reservo la memoria GPU.
cadena_ADN_char_GPU = cuda.mem_alloc( cadena_ADN_char.nbytes )
cadena_ADN_Complementaria_GPU = cuda.mem_alloc( cadena_ADN_Complementaria.nbytes )
cadena_ADN_Complementaria_Inversa_GPU = cuda.mem_alloc( cadena_ADN_Complementaria_Inversa.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( cadena_ADN_char_GPU, cadena_ADN_char )


# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_secADN( int n, char *origen, char *complementario, char *complementario_inversa )
{
  int indice = threadIdx.x + blockIdx.x*blockDim.x;
  if( indice < n )
  {
     if (origen[indice*4] =='A')
     {
        complementario[indice*4] = 'T';
     }else if(origen[indice*4] =='C')
    {
       complementario[indice*4] = 'G';
    }else if( origen[indice*4] =='G')
    {
        complementario[indice*4] = 'C';
    }else if(origen[indice*4] =='T' )
    {
        complementario[indice*4] = 'A' ;
    }
  }
}
""") 
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_secADN")

tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (cantidad_Nucleotidos+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(cantidad_Nucleotidos), cadena_ADN_char_GPU, cadena_ADN_Complementaria_GPU, cadena_ADN_Complementaria_Inversa_GPU, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( cadena_ADN_Complementaria, cadena_ADN_Complementaria_GPU )
cuda.memcpy_dtoh( cadena_ADN_Complementaria_Inversa, cadena_ADN_Complementaria_Inversa_GPU )

"""
# CPU - Informo el resutlado.
print( "------------------------------------")
print( "X: " )
print( x_cpu )
print( "------------------------------------")
print( "Y: " )
print( y_cpu )
print( "------------------------------------")
print( "R: " )
print( r_cpu )
"""

tiempo_total = datetime.now() - tiempo_total

print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Cadena ADN original: ", cadena_ADN_char )
print("Cadena ADN Complementaria: ", cadena_ADN_Complementaria )
#print("Cadena ADN Complementaria Inversa: ",  cadena_ADN_Complementaria_Inversa )
print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo bucle: ", tiempo_en_ms( tiempo_gpu ), "[ms]" )

Thread x:  256 , Bloque x: 1
Thread x:  256 , Bloque x: 1
Cadena ADN original:  ['A' 'C' 'T' 'G' 'A' 'A' 'A' 'T' 'C' 'G' 'C' 'G' 'G' 'C']
Cadena ADN Complementaria:  ['T' 'G' 'A' 'C' 'T' 'T' 'T' 'A' 'G' 'C' 'G' 'C' 'C' 'G']
Tiempo Total:  3.663 [ms]
Tiempo bucle:  0.592 [ms]


# 4 Tabla de pasos de ejecución del programa


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( Cantidad_N ) | Inicializa los vectoes A, B y R.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU      |  print()               | Informo los resultados.


# 5 Conclusiones.

Computacionalmente se recorre uno a uno los elementos de la cadena de ADN,  que realiza un algoritmo con complejidad N. Por cada elemento nuevo en la cadena, el tiempo de ejecucion es mas largo ya que se carga a la CPU con todo el procesamiento.


#6 Bibliografia
1 - Explicacion ADN: https://www.ncbi.nlm.nih.gov/books/NBK26821/

2 - Ejercicio Ejemplo Cadena ADN: https://www.youtube.com/watch?v=bcQQYsrGetM

3 - Informacion Cadenas ADN: https://es.wikipedia.org/wiki/Secuencia_de_ADN